In [1]:
import pandas as pd
from datetime import date
import math
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.preprocessing import MinMaxScaler
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout,LSTM,Activation
import tensorflow as tf

In [3]:
df = pd.read_csv('mgasoil.txt', sep=' ')

In [4]:
df.head()

,year,mon,Greg,hoil,oilp,gasp
0,1993,11,1.066,0.502,16.699,2.32
1,1993,12,1.014,0.435,14.510,2.09
2,1994,1,0.998,0.499,15.000,2.41
3,1994,2,1.009,0.557,14.780,2.59
4,1994,3,1.008,0.492,14.660,2.08


In [5]:
df['Date'] = df['year'].astype(str)+' /'+df['mon'].astype(str)

In [6]:
df_new = df[['Date', 'oilp']]

In [7]:
train_values = df_new.oilp.values.reshape(-1,1)
#train_values

In [8]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_dataset = scaler.fit_transform(train_values)

In [9]:
def create_dataset(scaled_data):
    window_size = 6
    x_set =[]
    y_set =[]
    
    for i in range(window_size,len(scaled_data)):
        x_set.append(scaled_data[i-window_size:i,:])
        y_set.append(scaled_data[i,0:1])
        
    return x_set, y_set

In [10]:
x_set, y_set = create_dataset(scaled_dataset)
x_data = np.array(x_set)
y_data = np.array(y_set)
print(x_data.shape)
print(y_data.shape)

(220, 6, 1)
(220, 1)


In [11]:
#Split data
#x_data
train_size = int(len(x_data)*0.8)
test_size_x = len(x_data) - train_size
train_data_x = x_data[:train_size]
test_data_x =  x_data[train_size:]

print(train_data_x.shape[1])

#y_data
train_size = int(len(y_data)*0.8)
test_size_y = len(y_data) - train_size
train_data_y = y_data[:train_size]
test_data_y =  y_data[train_size:]
print(train_data_y.shape)

6
(176, 1)


In [12]:
#LSTM model 
model = Sequential()

model.add(LSTM(units = 256, return_sequences = True, input_shape=(train_data_x.shape[1], train_data_x.shape[2])))
model.add(Dropout(0.2))

model.add(LSTM(units = 256, return_sequences = True))
model.add(Dropout(0.2))

#model.add(LSTM(units = 256, return_sequences = True))
#model.add(Dropout(0.2))

model.add(LSTM(units = 256))
model.add(Dropout(0.2))

model.add(Dense(units = 1))


model.compile(optimizer = 'adam', loss = 'mean_squared_error')  

In [13]:
history = model.fit(train_data_x, train_data_y, epochs = 200, batch_size=80)

Epoch 1/200
3/3 [==============================] - 5s 79ms/step - loss: 0.0766
Epoch 2/200
3/3 [==============================] - 0s 59ms/step - loss: 0.0176
Epoch 3/200
3/3 [==============================] - 0s 60ms/step - loss: 0.0194
Epoch 4/200
3/3 [==============================] - 0s 60ms/step - loss: 0.0157
Epoch 5/200
3/3 [==============================] - 0s 59ms/step - loss: 0.0088
Epoch 6/200
3/3 [==============================] - 0s 61ms/step - loss: 0.0122
Epoch 7/200
3/3 [==============================] - 0s 60ms/step - loss: 0.0077
Epoch 8/200
3/3 [==============================] - 0s 60ms/step - loss: 0.0098
Epoch 9/200
3/3 [==============================] - 0s 68ms/step - loss: 0.0088
Epoch 10/200
3/3 [==============================] - 0s 64ms/step - loss: 0.0083
Epoch 11/200
3/3 [==============================] - 0s 60ms/step - loss: 0.0081
Epoch 12/200
3/3 [==============================] - 0s 66ms/step - loss: 0.0080
Epoch 13/200
3/3 [==============================]

3/3 [==============================] - 0s 97ms/step - loss: 0.0036
Epoch 104/200
3/3 [==============================] - 0s 99ms/step - loss: 0.0019
Epoch 105/200
3/3 [==============================] - 0s 81ms/step - loss: 0.0040
Epoch 106/200
3/3 [==============================] - 0s 79ms/step - loss: 0.0018
Epoch 107/200
3/3 [==============================] - 0s 101ms/step - loss: 0.0020
Epoch 108/200
3/3 [==============================] - 0s 86ms/step - loss: 0.0022
Epoch 109/200
3/3 [==============================] - 0s 88ms/step - loss: 0.0018
Epoch 110/200
3/3 [==============================] - 0s 95ms/step - loss: 0.0023
Epoch 111/200
3/3 [==============================] - 0s 74ms/step - loss: 0.0037
Epoch 112/200
3/3 [==============================] - 0s 74ms/step - loss: 0.0027
Epoch 113/200
3/3 [==============================] - 0s 76ms/step - loss: 0.0034
Epoch 114/200
3/3 [==============================] - 0s 75ms/step - loss: 0.0025
Epoch 115/200
3/3 [======================

In [24]:
y_predicted = model.predict(test_data_x)
y_predicted.shape

2/2 [==============================] - 0s 14ms/step


(44, 1)

In [25]:
#Calculate RMSE
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(scaler.inverse_transform(test_data_y), scaler.inverse_transform(y_predicted))

rmse = np.sqrt(mse)
rmse

6.627922622246155